### Imports

In [1]:
from typing import List, Tuple
from datetime import datetime
from trade_flow.environments import metatrader
from trade_flow.environments.metatrader import Timeframe, FOREX_DATA_PATH, STOCKS_DATA_PATH, CRYPTO_DATA_PATH, MIXED_DATA_PATH

/home/fortesenselabs/Tech/labs/Financial_Eng/Financial_Markets/lab/trade_flow/trade_flow/feed/__init__.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(path, parse_dates=True, index_col=index_name)


In [2]:
def get_data(symbols: List[str] = ["EURUSD", "GBPCAD", "USDJPY"], 
             time_range: Tuple[datetime, datetime] = (datetime(2011, 1, 1), datetime(2012, 12, 31)),
             timeframe: Timeframe = Timeframe.D1, 
             filename: str = FOREX_DATA_PATH):
    
    mt_sim = metatrader.Simulator()
    mt_sim.download_data(symbols, time_range, timeframe)
    mt_sim.save_symbols(filename)


In [3]:
# Define the time range for the data download
start_date = datetime(2016, 1, 1)
end_date = datetime(2020, 12, 31)
time_range = (start_date, end_date)

In [4]:
# forex
get_data(["EURUSD", "GBPCAD", "USDJPY"], time_range, Timeframe.D1, FOREX_DATA_PATH)

In [5]:
# stocks
get_data(["GOOG", "AAPL", "TSLA", "MSFT"], time_range, Timeframe.D1, STOCKS_DATA_PATH)

In [6]:
# crypto
get_data(["BTCUSD", "ETHUSD", "BCHUSD"], time_range, Timeframe.D1, CRYPTO_DATA_PATH)

In [7]:
# mixed 
get_data(["EURUSD", "USDCAD", "GOOG", "AAPL", "BTCUSD", "ETHUSD"], time_range, Timeframe.D1, MIXED_DATA_PATH)